<a href="https://colab.research.google.com/github/KNGLJordan/CDMO-project/blob/main/sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install z3
!pip install z3-solver

from z3 import *
from itertools import combinations

In [3]:
#the man the myth the legend: angelo quarta
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

In [12]:
from z3 import *
from itertools import combinations

#the man the myth the legend: angelo quarta
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

# === PARAMETERS ===
n = 6
weeks = n - 1
periods = n // 2
teams = list(range(n))  # 0-based indexing

solver = Optimize()

# === VARIABLES ===
# plays_home[t][w][p] = True if team t plays home in week w, period p
# plays_away[t][w][p] = True if team t plays away in week w, period p
plays_home = [[[Bool(f"home_team{t}_week{w}_period{p}") for p in range(periods)] for w in range(weeks)] for t in teams]
#if n = 6, therefore weeks = 5, periods = 3, then this will create a 3D list of size 6x5x3, of boolean variables
#for example: plays_home[0][1][2] = Bool("home_team0_week1_period2") = True if team 0 plays home in week 1, period 2
#in general: plays_away[t][w][p] = True if team t plays away in week w, period p
plays_away = [[[Bool(f"away_team{t}_week{w}_period{p}") for p in range(periods)] for w in range(weeks)] for t in teams]

# === 1. Every team plays once per week ===
for t in teams:
    for w in range(weeks):
        vars_in_week = [plays_home[t][w][p] for p in range(periods)] + [plays_away[t][w][p] for p in range(periods)]
        solver.add(exactly_one_np(vars_in_week, f"play_once_{t}_{w}"))

# Each period (w, p) must have exactly one match: one home + one away
for w in range(weeks):
    for p in range(periods):
        home_teams = [plays_home[t][w][p] for t in teams] #in every iteration, home_teams will be a list of home teams that play in week w and period p
        away_teams = [plays_away[t][w][p] for t in teams] #same thing
        solver.add(exactly_one_np(home_teams, f"one_home_w{w}_p{p}")) #we want to have exactly one home team in week w and period p
        solver.add(exactly_one_np(away_teams, f"one_away_w{w}_p{p}"))

# === 2. Every pair of teams plays exactly once ===
# We'll create match_ij[w][p] = Bool if i vs j is scheduled in that week/period
for i in teams:
    for j in teams:
        if i >= j:
            continue
        match_conditions = [] #For each pair, we create an empty list match_conditions
        for w in range(weeks):
            for p in range(periods):
                # i home, j away OR j home, i away
                match_conditions.append(
                    Or(
                        And(plays_home[i][w][p], plays_away[j][w][p]),
                        And(plays_home[j][w][p], plays_away[i][w][p])
                    )
                )
        # Use exactly_one_np instead of Sum == 1
        solver.add(exactly_one_np(match_conditions, f"one_match_{i}_{j}"))

# === 3. A team plays at most twice in the same period (across weeks) ===
for team in teams:
    for period in range(periods):
        games_in_this_period = 0

        for week in range(weeks):
            # Add 1 if team plays at home in this week/period
            games_in_this_period += plays_home[team][week][period]
            # Add 1 if team plays away in this week/period
            games_in_this_period += plays_away[team][week][period]

        # Ensure total is at most 2
        solver.add(games_in_this_period <= 2)

# === 4. Optional: Home/Away balance ===
# Store home/away counts for each team (needed later for printing)
home_counts = []  # List to store home count expressions
away_counts = []  # List to store away count expressions
imbalances = []   # List to store imbalance variables

for t in teams:
    # Count home games for team t
    home_count = Sum([plays_home[t][w][p] for w in range(weeks) for p in range(periods)])
    # Count away games for team t
    away_count = Sum([plays_away[t][w][p] for w in range(weeks) for p in range(periods)])

    # Store these for later use in printing
    home_counts.append(home_count)
    away_counts.append(away_count)

    # Create imbalance variable for team t
    imbalance = Int(f"imbalance_{t}")
    imbalances.append(imbalance)

    # Add constraint: imbalance = |home_count - away_count|
    solver.add(imbalance == If(home_count >= away_count,
                              home_count - away_count,
                              away_count - home_count))

# Minimize the maximum imbalance across all teams
max_imbalance = Int("max_imbalance")
for imbalance in imbalances:
    solver.add(max_imbalance >= imbalance)

# Optional: ensure max_imbalance is tight (equals the actual maximum)
solver.add(Or([max_imbalance == imbalance for imbalance in imbalances]))

solver.minimize(max_imbalance)

# === SOLVE ===
if solver.check() == sat:
    model = solver.model()
    print("Schedule:")
    for w in range(weeks):
        print(f"Week {w + 1}:")
        for p in range(periods):
            for i in teams:
                if model.evaluate(plays_home[i][w][p], model_completion=True):
                    for j in teams:
                        if model.evaluate(plays_away[j][w][p], model_completion=True):
                            print(f"  Period {p + 1}: Team {i + 1} (home) vs Team {j + 1} (away)")

    print("\nImbalances:")
    for t in teams:
        h = model.evaluate(home_counts[t])  # Use home_counts[t] instead of home_count[t]
        a = model.evaluate(away_counts[t])  # Use away_counts[t] instead of away_count[t]
        d = model.evaluate(imbalances[t])   # Use imbalances[t] instead of imbalances
        print(f"Team {t+1}: Home={h}, Away={a}, Δ={d}")
    print("Max imbalance:", model.evaluate(max_imbalance))
else:
    print("No solution found.")

Schedule:
Week 1:
  Period 1: Team 4 (home) vs Team 2 (away)
  Period 2: Team 5 (home) vs Team 6 (away)
  Period 3: Team 1 (home) vs Team 3 (away)
Week 2:
  Period 1: Team 3 (home) vs Team 5 (away)
  Period 2: Team 6 (home) vs Team 2 (away)
  Period 3: Team 4 (home) vs Team 1 (away)
Week 3:
  Period 1: Team 6 (home) vs Team 1 (away)
  Period 2: Team 3 (home) vs Team 4 (away)
  Period 3: Team 2 (home) vs Team 5 (away)
Week 4:
  Period 1: Team 4 (home) vs Team 5 (away)
  Period 2: Team 1 (home) vs Team 2 (away)
  Period 3: Team 3 (home) vs Team 6 (away)
Week 5:
  Period 1: Team 2 (home) vs Team 3 (away)
  Period 2: Team 5 (home) vs Team 1 (away)
  Period 3: Team 6 (home) vs Team 4 (away)

Imbalances:
Team 1: Home=2, Away=3, Δ=1
Team 2: Home=2, Away=3, Δ=1
Team 3: Home=3, Away=2, Δ=1
Team 4: Home=3, Away=2, Δ=1
Team 5: Home=2, Away=3, Δ=1
Team 6: Home=3, Away=2, Δ=1
Max imbalance: 1
